In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install anesthetic
import os
os.chdir("/content/drive/MyDrive/project")

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import torch
import torch.nn as nn
import torch.distributions as dists
import torch.optim as optim
from torch.distributions import MultivariateNormal
from torch.distributions.distribution import Distribution
from np_utils import simple_data, curved_data, planck_des_data
from torch_utils import rotation_test, get_limits, TrainUtil
from tension_net import TensionNet, TensionNet1, TensionNet2, TensionNet3
from tension_quantify import GaussianKDE, BayesFactorKDE, BayesFactor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
params = ["omegabh2", "omegam", "H0", "tau", "sigma8", "ns"]
param_pairs = []
for i in range(len(params)):
    for j in range(i + 1, len(params)):
        param_pairs.append([i, j])

In [ ]:
(X0, X0_weights, X1, X1_weights, X_prior, X_prior_weights,
 params, param_means, norm_factors) = planck_des_data(
     params=params, div_max=True
)
weights = { "XA": X0_weights, "XB": X1_weights, "X_prior": X_prior_weights }
param_means = torch.tensor(param_means).float().to(device)
norm_factors = torch.tensor(norm_factors).float().to(device)

In [ ]:
for pair in param_pairs:
    losses = []
    X0_r = np.delete(X0, pair, axis=1)
    X1_r = np.delete(X1, pair, axis=1)
    X_prior_r = np.delete(X_prior, pair, axis=1)

    for i in range(5):
        start = time.time()
        tension_R_cosmo = TensionNet1(4, hidden_size=4096)
        criterion = BayesFactorKDE(device, n_points=500)
        optimizer = optim.Adam(tension_R_cosmo.parameters(), lr=0.0001)

        train_util_R_cosmo = TrainUtil(tension_R_cosmo, optimizer, criterion, device)
        losses_R_cosmo = train_util_R_cosmo.train(X0_r, X1_r, X_prior_r, weights=weights,
                                                        n_iter=500, decrease_lr_at=[-8])
        losses.append(losses_R_cosmo[-1])
    
    print(pair, losses)

Histogram time taken:  74.174809217453 Final Loss:  -11.91533088684082
Histogram time taken:  73.92773294448853 Final Loss:  -2.0214080810546875
Histogram time taken:  73.87825202941895 Final Loss:  -11.775236129760742
Histogram time taken:  74.01312780380249 Final Loss:  -12.01418399810791
